https://www.youtube.com/watch?v=xdq6Gz33khQ&list=TLPQMTMwMTIwMjJO8mOZGdqcXA&index=4

In [1]:
!pip install requests

In [3]:
import requests
import datetime

import base64

In [4]:
client_id = '8e6e962237b74490aca58fd858c5a54f'
client_secret = '06fd25cc7b164f7e88927938c5c1a589'

In [10]:
# This is now a reusable Spotify API object

class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret

    def get_client_credentials(self):
        """
        Returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}"
        }
    
    def get_token_data(self):
        return {
            "grant_type": "client_credentials"
        } 
    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200, 299):
            return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
        

In [6]:
spotify = SpotifyAPI(client_id, client_secret)

In [7]:
spotify.perform_auth()

True

In [8]:
spotify.access_token

'BQCOgtCIiLe5ZI4CVXRYHp04Zo6UoVK8GRY-Zig_Ix1D4i_wzFlBhcu9hyr6Z7FNDA3XnX0AmIj5Xb7uYX4'

In [9]:
# spotify.search